# Lab 4 DL with Keras

### Step 1 Import all necessary packages

In [ ]:
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid, train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import Callback, TensorBoard
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tqdm import tqdm

### Step 2 Load and preprocess the data

In [ ]:
# TODO: Load the Iris dataset or any other dataset of your choice

In [ ]:
# TODO: Rescale the data

# TODO: Determine the number of components to keep for PCA

In [ ]:
# TODO: Apply PCA to the data to reduce the dimensionality

### Step 3 Define a custom Tensorboard callback for saving training results

In [ ]:
class CustomTensorBoardCallback(Callback):
    def __init__(self, log_dir):
        super().__init__()
        self.log_dir = log_dir

    def on_train_begin(self, logs=None):
        self.model.tensorboard_callback = TensorBoard(
            log_dir=self.log_dir, histogram_freq=1)
        self.model.tensorboard_callback.set_model(self.model)

    def on_epoch_end(self, epoch, logs=None):
        self.model.tensorboard_callback.on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        self.model.tensorboard_callback.on_train_end(logs)

### Step 4 Create an ANN with Keras

In [ ]:
# TODO: Create the neural network model with specified hyperparameters
def create_model(n_units1=64, n_units2=32, dropout_rate=0.5, l1_l2_reg=(1e-5, 1e-4)):
    model = None
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Wrap the model with KerasClassifier to use it with scikit-learn API
model = KerasClassifier(build_fn=create_model,
                        epochs=100, batch_size=32, verbose=0)

### Step 5 Define the parameters you want to tune

In [ ]:
# Grid search hyperparameters
param_grid = {
    'n_units1': [32, 64],
    'n_units2': [16, 32],
    'dropout_rate': [0.25, 0.5],
    'l1_l2_reg': [(1e-5, 1e-4), (1e-4, 1e-3)]
}
grid = ParameterGrid(param_grid)

### Step 6 Setup the logging with Tensorboard and train for each set of parameters

In [ ]:
# Define the logs directory
logs_dir = "logs"

# Remove the logs directory if it exists
if os.path.exists(logs_dir):
    shutil.rmtree(logs_dir)

# Create a new logs directory
os.makedirs(logs_dir)

In [ ]:
best_score = -np.inf
best_params = None

# Perform the grid search
i = 0
for params in tqdm(grid):
    print("Training for the following params:", params)
    log_dir = f"logs/grid_search_{i}"
    tensorboard_callback = CustomTensorBoardCallback(log_dir=log_dir)
    model.set_params(**params)
    model.fit(X_train, y_train, callbacks=[tensorboard_callback])
    score = model.score(X_test, y_test)

    if score > best_score:
        best_score = score
        best_params = params

    i += 1


In [ ]:
print("Best hyperparameters:", best_params)

### Step 7 Create a new model with the best found parameter set and evaluate the model

In [ ]:
best_model = create_model(**best_params)
best_model.compile(
    optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
best_model.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")